In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import cv2

# Training and pre-processing

loading the csv

In [5]:
data = pd.read_csv('fer2013.csv')
images = data['pixels']
labels = data['emotion']

Preprocssing



In [6]:
def preprocess_images(images):
    images = np.array([np.fromstring(image, dtype=int, sep=' ').reshape(48, 48) for image in images])
    images = np.expand_dims(images, axis=-1)
    images = images / 255.0
    return images

images = preprocess_images(images)


Split the Dataset

In [7]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)


Define the CNN Model

In [8]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')
])


Compile and Train the Model

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
898/898 [==============================] - 29s 31ms/step - loss: 1.5963 - accuracy: 0.3724 - val_loss: 1.4868 - val_accuracy: 0.4205
Epoch 2/10
898/898 [==============================] - 26s 29ms/step - loss: 1.3315 - accuracy: 0.4939 - val_loss: 1.2619 - val_accuracy: 0.5251
Epoch 3/10
898/898 [==============================] - 28s 32ms/step - loss: 1.2188 - accuracy: 0.5391 - val_loss: 1.2395 - val_accuracy: 0.5209
Epoch 4/10
898/898 [==============================] - 27s 30ms/step - loss: 1.1409 - accuracy: 0.5695 - val_loss: 1.2034 - val_accuracy: 0.5357
Epoch 5/10
898/898 [==============================] - 26s 29ms/step - loss: 1.0779 - accuracy: 0.5955 - val_loss: 1.1774 - val_accuracy: 0.5522
Epoch 6/10
898/898 [==============================] - 27s 31ms/step - loss: 1.0152 - accuracy: 0.6195 - val_loss: 1.1619 - val_accuracy: 0.5602
Epoch 7/10
898/898 [==============================] - 26s 29ms/step - loss: 0.9541 - accuracy: 0.6440 - val_loss: 1.1848 - val_accuracy:

Capture photo

load the Haar cascade

In [10]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


Define the emotions

In [11]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Predit funcition

In [12]:
def predict_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = np.expand_dims(face, axis=-1)
        face = np.expand_dims(face, axis=0)
        prediction = model.predict(face)
        emotion = emotions[np.argmax(prediction)]
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    return frame


In [16]:
cap = cv2.VideoCapture(0)

Test on live video

In [17]:
while True:
    ret, frame = cap.read()
    if ret:
        frame = predict_emotion(frame)
        cv2.imshow('Live Facial Emotion Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 18ms/step


KeyboardInterrupt: 